In [2]:
!pip install openai pandas

In [3]:
import pandas as pd
import time
import openai

In [4]:


# Set up OpenAI API credentials
openai.api_key = '<YOUR-KEY>'


LLAMA = 'Llama'
DATABRICKS_DOLLY = 'Dolly'
CHAT_GPT = 'ChatGPT'

# Set up the prompts
english_prompts = [
    "What are some popular beaches in Brazil?",
    "Tell me about the marine life along the Brazilian coast.",
    "What activities can you do on the Brazilian coastline?",
    "Tell me all you know about brazilian coast.",
    "Some species that inhabit the brazilian coast are",
    
]

portuguese_prompts = [
    "Quais são algumas praias populares no Brasil?",
    "Fale sobre a vida marinha ao longo da costa brasileira.",
    "Quais atividades você pode fazer na costa brasileira?",
    "Me fale tudo o que você sabe sobre a costa brasileira.",
    "Algumas espécies que habitam a costa brasileira são",
]

# Create a pandas dataframe to store the benchmark results
df = pd.DataFrame(columns=['model', 'prompt', 'language',
                           'generated_response', 'time_elapsed'])

# Create a pandas dataframe to store the benchmark results
df = pd.DataFrame(columns=['model', 'prompt', 'language',
                           'generated_response', 'time_elapsed'])

# Define a function to run the benchmark for a given model and prompts
def run_benchmark(model, prompts, language):
    for i, prompt in enumerate(prompts):
        start_time = time.time()

        if model == 'ChatGPT':
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo', 
                messages=[
                        {
                            "role": "system",
                             "content": "You are a helpful assistant."
                        },
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],
                  
                 
                 
            )
            
            response = response.choices[0].message.to_dict()['content']
            

        elif model == 'Llama':
            import torch

            # free cuda resources being used by another possible models
            torch.cuda.empty_cache()
            
            if 'generate_text' in globals():
                del generate_text
            
             

        elif model == 'Databricks Dolly':
            
            if 'generate_text' not in globals():
                generate_text = pipeline(model="databricks/dolly-v2-3b",
                            torch_dtype=torch.bfloat16,
                            trust_remote_code=True,
                            device_map="auto")
            
            response = generate_text(prompt)[0]['generated_text']

        elapsed_time = time.time() - start_time

        df.loc[len(df)] = [model, prompt, language, response, elapsed_time]
        
        # avoid hitting the openAPI rate limit
        time.sleep(20)
        
        print(f'Finished prompt {prompt} for model {model} in {language}')
         

# Run the benchmark for ChatGPT
run_benchmark('ChatGPT', english_prompts, 'en')
run_benchmark('ChatGPT', portuguese_prompts, 'pt')

# Run the benchmark for Llama
# run_benchmark('Llama', english_prompts, 'en')
# run_benchmark('Llama', portuguese_prompts, 'pt')

# # Run the benchmark for Databricks Dolly
# run_benchmark('Databricks Dolly', english_prompts, 'en')
# run_benchmark('Databricks Dolly', portuguese_prompts, 'pt')

df

,Model,Prompt,Response,Time
0,ChatGPT,What are some popular beaches in Brazil?,There are many popular beaches in Brazil. Some...,3.693583
1,ChatGPT,Tell me about the marine life along the Brazil...,The marine life along the Brazilian coast is d...,9.172756
2,ChatGPT,What activities can you do on the Brazilian co...,There are many activities that you can enjoy o...,11.106159
3,ChatGPT,Tell me all you know about brazilian coast.,"Sure, the Brazilian coast is one of the longes...",10.458329
4,ChatGPT,Some species that inhabit the brazilian coast are,:\n\n1. Green Sea Turtles\n2. Humpback Whales\...,4.386691
5,ChatGPT,Quais são algumas praias populares no Brasil?,Algumas praias populares no Brasil são:\n\n1. ...,8.665694
6,ChatGPT,Fale sobre a vida marinha ao longo da costa br...,A costa brasileira possui uma rica vida marinh...,11.482349
7,ChatGPT,Quais atividades você pode fazer na costa bras...,"Na costa brasileira, há muitas atividades para...",16.725192
8,ChatGPT,Me fale tudo o que você sabe sobre a costa bra...,A costa brasileira tem cerca de 8.500 km de ex...,15.971384
9,ChatGPT,Algumas espécies que habitam a costa brasileir...,Algumas das espécies que habitam a costa brasi...,6.045625


In [ ]:
df['Model'] = df['model'].map({'ChatGPT': 'ChatGPT', 'Llama': 'Llama', 'Databricks Dolly': 'Dolly'})

In [5]:
df.to_csv('dumb_benchmark.csv')